## PMX - Phuture Meme Coin Index- Methodology Template

## Configuring packages

In [16]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
sys.path.append('../')
import datetime 
import time

In [17]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [18]:
import plotly.express as px
import plotly.graph_objects as go

In [19]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables

In [20]:
min_mcap =  1.5e8
min_weight = 0.005
max_weight = None
circ_supply_threshold = 0.3
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "meme-token"


In [21]:
# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)
max_slippage

-0.04

## Data retrieval and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $150mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


#### Supported assets and blockchains

In [22]:
# Stable coins to remove from asset list
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')
# Stargate supported blockchains key value pairing: key = blockchain identifier, value = native token identifier
sg_blockchains =  {
    'ethereum': 'ethereum',
    'avalanche': 'avalanche-2',
    'binance-smart-chain': 'binancecoin',
    'polygon-pos': 'matic-network',
    'arbitrum-one': 'ethereum',
    'arbitrum-nova': 'ethereum',
    'fantom': 'fantom',
    'optimistic-ethereum': 'ethereum'
}

#### CoinGecko

##### Gathering market data

In [23]:

def get_category_data(category_id, min_mcap):
    coin_market_data = pd.DataFrame(cg.get_coins_markets('usd',category=category_id,order='market_cap_desc',per_page = 250))
    # Removing tokens with a market cap below the threshold
    coin_market_data = coin_market_data[coin_market_data['market_cap']>=min_mcap]
    coin_market_data.set_index('id',inplace=True)
    coin_market_data = coin_market_data[['symbol','name','current_price','market_cap','market_cap_rank','fully_diluted_valuation','circulating_supply','total_supply','max_supply']]
    return coin_market_data

def add_asset_to_category(data,ids):
    coin_data = pd.DataFrame(cg.get_coins_markets('usd',ids=ids))
    coin_data.set_index('id',inplace=True)
    coin_data = coin_data[['symbol','name','current_price','market_cap','market_cap_rank','fully_diluted_valuation','circulating_supply','total_supply','max_supply']]
    data = pd.concat([data,coin_data])
    data.sort_values(by=['market_cap'],inplace=True,ascending=False)
    return data

def replace_id(category_data,ids,replacement_ids):
    for i in range(len(id)):
        category_data.rename(index={ids[i]:replacement_ids[i]},inplace=True)
    return category_data

def get_all_coin_data():
    coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))
    coins_list.set_index('id',inplace=True)
    return coins_list

def filter_coin_data(all_coin_data,category_data, df_to_remove):
    all_coin_data.query('index in @category_data.index',inplace=True)
    for df in df_to_remove:
        all_coin_data.drop(df.index,inplace=True, errors='ignore')
    for id, data in all_coin_data.iterrows():
        platforms = list(data['platforms'].keys())
        to_remove = True
        for blockchain in platforms:
            if blockchain in sg_blockchains.keys():
                to_remove = False
        if id in sg_blockchains.values():
            to_remove = False
        if to_remove  == True:
            all_coin_data.drop(id,inplace=True)
    return all_coin_data

def merge_data(category_data, filtered_data):
    category_data = category_data.join(filtered_data['platforms'],how='inner',on='id')
    return category_data

category_data = get_category_data(coingecko_category,min_mcap)
coin_data = get_all_coin_data()
coin_data = filter_coin_data(coin_data,category_data,[stablecoins,ctokens,atokens])
coin_data = merge_data(category_data,coin_data)
coin_data

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply,platforms
id,,,,,,,,,,
shiba-inu,shib,Shiba Inu,8.010000e-06,4.717102e+09,20.0,8.004082e+09,5.893266e+14,9.999824e+14,NaN,{'ethereum': '0x95ad61b0a150d79219dcf64e1e6cc0...
pepe,pepe,Pepe,1.090000e-06,4.553274e+08,108.0,4.553274e+08,4.206900e+14,4.206900e+14,4.206900e+14,{'ethereum': '0x6982508145454ce325ddbe47a25d4e...
floki,floki,FLOKI,2.940000e-05,2.911192e+08,145.0,2.928177e+08,9.941994e+12,1.000000e+13,1.000000e+13,{'ethereum': '0xcf0c122c6b73ff809c693db761e7ba...
memecoin-2,meme,Memecoin,2.441842e-02,1.835056e+08,200.0,1.678044e+09,7.545625e+09,6.900000e+10,6.900000e+10,{'ethereum': '0xb131f4a55907b10d1f0a50d8ab8fa0...
baby-doge-coin,babydoge,Baby Doge Coin,1.207000e-09,1.766604e+08,206.0,5.070517e+08,1.463310e+17,4.200000e+17,4.200000e+17,{'binance-smart-chain': '0xc748673057861a79727...
bonk,bonk,Bonk,3.630000e-06,1.546577e+08,226.0,3.377942e+08,4.297493e+13,9.386328e+13,9.386328e+13,{'solana': 'DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7...


### Token's supply and asset maturity checks

In [24]:
def token_supply_check(data, threshold):
    supply_check = coin_data['circulating_supply']/ coin_data['total_supply'] > threshold
    return data[supply_check]

def asset_maturity_check(data, threshold):
    prices_data = pd.DataFrame()
    marketcaps = pd.DataFrame()
    for id,_ in data.iterrows():
        cg_data = cg.get_coin_market_chart_by_id(id, vs_currency='USD', days='max')
        df_prices = pd.DataFrame(cg_data['prices'], columns=['date', id])
        df_prices = df_prices[df_prices[id] > 0]
        df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
        df_prices['date'] = pd.to_datetime(df_prices['date'])
        df_prices = df_prices.set_index('date', drop=True)
        df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
        
        if len(df_prices) < threshold:
            print(f'Excluding {id}, pricing data available only for {len(df_prices)} < {liveness_threshold} days')
            data.drop(id,inplace=True)
        else:
            prices_data = pd.concat([prices_data, df_prices], axis=1)
            
            df_mcaps = pd.DataFrame(cg_data['market_caps'], columns=['date', id])
            df_mcaps = df_mcaps[df_mcaps[id] > 0]
            df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
            df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
            df_mcaps = df_mcaps.set_index('date', drop=True)
            df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
            
            if len(df_mcaps) < liveness_threshold:
                print(f'Note: {id}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
            df_mcaps = df_mcaps.reindex(df_prices.index)
            marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    return (data, prices_data,marketcaps)

coin_data,historical_pricing,historical_mcaps  = asset_maturity_check(token_supply_check(coin_data,circ_supply_threshold),liveness_threshold)


### Non quantative checks

In [25]:
manual_exclusions = []

def remove_manual_exclusions(data,exclusion_list):
    for id in exclusion_list:
        data.drop(id,inplace=True)
    return data
coin_data = remove_manual_exclusions(coin_data,manual_exclusions)
coin_data

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply,platforms
id,,,,,,,,,,
shiba-inu,shib,Shiba Inu,8.010000e-06,4.717102e+09,20.0,8.004082e+09,5.893266e+14,9.999824e+14,NaN,{'ethereum': '0x95ad61b0a150d79219dcf64e1e6cc0...
pepe,pepe,Pepe,1.090000e-06,4.553274e+08,108.0,4.553274e+08,4.206900e+14,4.206900e+14,4.206900e+14,{'ethereum': '0x6982508145454ce325ddbe47a25d4e...
floki,floki,FLOKI,2.940000e-05,2.911192e+08,145.0,2.928177e+08,9.941994e+12,1.000000e+13,1.000000e+13,{'ethereum': '0xcf0c122c6b73ff809c693db761e7ba...
baby-doge-coin,babydoge,Baby Doge Coin,1.207000e-09,1.766604e+08,206.0,5.070517e+08,1.463310e+17,4.200000e+17,4.200000e+17,{'binance-smart-chain': '0xc748673057861a79727...
bonk,bonk,Bonk,3.630000e-06,1.546577e+08,226.0,3.377942e+08,4.297493e+13,9.386328e+13,9.386328e+13,{'solana': 'DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7...


### Liquidity check

In [26]:
# URLs for 0x
url_0x = {
    'ethereum' : 'https://api.0x.org/swap/v1/quote',
    'polygon-pos' : 'https://polygon.api.0x.org/swap/v1/quote',
    'binance-smart-chain' : 'https://bsc.api.0x.org/swap/v1/quote',
    'optimistic-ethereum' : 'https://optimism.api.0x.org/swap/v1/quote',
    'fantom' : 'https://fantom.api.0x.org/swap/v1/quote',
    'avalanche' : 'https://avalanche.api.0x.org/swap/v1/quote',
    'arbitrum-nova' : 'https://arbitrum.api.0x.org/swap/v1/quote',
    'arbitrum-one' : 'https://arbitrum.api.0x.org/swap/v1/quote',
}
header =  {'0x-api-key': decouple.config("ZEROEX_KEY")}

In [27]:
stablecoin_by_blockchain_info = {
    'ethereum':
        {'address':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48','decimals':6},
    'avalanche':
        {'address':'0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E','decimals': 6},
    'polygon-pos':
        {'address': '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174','decimals': 6},
    'arbitrum-nova':
        {'address': '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8','decimals': 6},
    'arbitrum-one':
        {'address': '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8','decimals': 6},
    'optimistic-ethereum':
        {'address': '0x7F5c764cBc14f9669B88837ca1490cCa17c31607','decimals': 6},
    'fantom':
        {'address': '0x04068DA6C83AFCFA0e13ba15A6696662335D5B75','decimals': 6},
    'binance-smart-chain':
        {'address': '0x8AC76a51cc950d9822D68b83fE1Ad97B32Cd580d', 'decimals': 18}        
}

In [28]:
def calculate_slippage(buy_token, blockchain):
    decimals = stablecoin_by_blockchain_info[blockchain]['decimals']
    sell_token_id = "usd-coin"
    try:
        query = {
            'buyToken': buy_token,
            'sellToken': stablecoin_by_blockchain_info[blockchain]['address'],
            'sellAmount': int(trade_value_tiny / cg.get_price(sell_token_id,'usd')[sell_token_id]['usd']) * 10 ** decimals,
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x[blockchain], params=query,headers=header)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / cg.get_price(sell_token_id,'usd')[sell_token_id]['usd']) * 10 ** decimals
        time.sleep(1.5)
        resp = requests.get(url_0x[blockchain], params=query,headers=header)
        swap = resp.json()
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1

        return {'spot price':spot_price, 'delivery price' : del_price,'slippage' :slippage, 'blockchain': blockchain}
    
    except KeyError:
        print(buy_token)
        return None    

In [29]:
def get_blockchain_by_native_asset(coin_id):
    for blockchain, native_asset in sg_blockchains.items():
        if coin_id == native_asset:
            return blockchain
    return None 

In [30]:
def assess_liquidity(data,threshold):
    slippages = []
    # Iterate over each row of the dataframe
    for id, coin_data in data.iterrows():
        slippage_dict = {'slippage': float('-inf')}
        # If there are no platforms listed it is likely a native asset so we use symbol instead of address for the buy token
        if len(coin_data['platforms'].keys()) == 0:
            slippage_dict = calculate_slippage(coin_data['symbol'].upper(),get_blockchain_by_native_asset(id))
            # If response is not None then we replace the current slippage dictionary with the return one
            if slippage_dict is not None:
                slippage_dict['id'] = id
                slippages.append(slippage_dict)
            else:
                continue
        else:
            # Iterate over each blockchain the asset is listed on     
            for blockchain in coin_data['platforms'].keys():
                # Check that the blockchain is supported
                if blockchain in sg_blockchains.keys():
                    temp_slippage_dict = calculate_slippage(coin_data['platforms'][blockchain],blockchain)
                    # If response is not None and the return slippage is less negative than what is stored in slippage_dict then replace
                    if temp_slippage_dict is not None and temp_slippage_dict['slippage'] > slippage_dict['slippage']:
                        temp_slippage_dict['id'] = id
                        slippage_dict = temp_slippage_dict

                    else:
                        continue
                else:
                    continue
            # Check whether asset is native to a supported blockchain
            blockchain = get_blockchain_by_native_asset(id)
            if blockchain is not None:
                temp_slippage_dict = calculate_slippage(coin_data['symbol'],blockchain)
                # If return slippage is less negative than what is stored in slippage_dict then replace
                if temp_slippage_dict is not None and temp_slippage_dict['slippage'] > slippage_dict['slippage']:
                    temp_slippage_dict['id'] = id
                    slippage_dict = temp_slippage_dict   
            # If length of slippage_dict is greater than 1 this means there is a valid response to store        
            if len(slippage_dict) > 1:    
                slippages.append(slippage_dict)
            # Else slippage_dict stores the default value and thus no valid response has been stored
            else:
                continue
    slippage_pd = pd.DataFrame(slippages).set_index('id')
    return (data[slippage_pd['slippage'] > threshold],slippage_pd)

coin_data,slippage_data = assess_liquidity(coin_data,max_slippage)
slippage_data       

0xc748673057861a797275cd8a068abb95a902e8de
0xe5b49820e5a1063f6f4ddf851327b5e8b2301048


,spot price,delivery price,slippage,blockchain
id,,,,
shiba-inu,1.246649e+05,1.222568e+05,-0.019317,ethereum
pepe,9.196229e+05,9.144458e+05,-0.005630,ethereum
floki,3.395734e+04,3.300021e+04,-0.028186,ethereum
baby-doge-coin,7.387594e+08,6.268916e+08,-0.151427,ethereum
bonk,2.659185e+05,2.335316e+04,-0.912179,ethereum


### Marketcap ranking & filtering

In [31]:
prices_data = historical_pricing[coin_data.index]
marketcaps = historical_mcaps[coin_data.index]


In [32]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

### Weight mcaps

In [33]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [34]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [35]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

def compute_max_weight(data):
    global max_weight
    max_weight = 1/len(data)

In [36]:
compute_max_weight(coin_data)
adjusted_weights = adjust_weights(remove_tiny_weights(adjust_weights(weights, max_weight),min_weight),max_weight)
adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)

In [37]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [38]:
def convert_weights(weights):
    max = np.floor(max_weight *255)
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)
    remainders = w_scaled - w_res
    k = round(remainders.sum())
    w_below_max = w_res[w_res<max]
    while k > 0:
        for i in range(w_below_max):
            w_below_max[i] +=1
            k -= 1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

last_weights = adjusted_weights.iloc[-1].dropna()
converted_last_weights = convert_weights(last_weights)

## Summary table

In [39]:
result = pd.DataFrame()
result.index = coin_data.index
result['name'] = coin_data['name']
result['market_cap'] = marketcaps.iloc[-1].astype(int)
result['price'] = prices_data.iloc[-1]
result['weight'] = last_weights
result['weight_converted'] = converted_last_weights
result['address'] = [data['platforms'][slippage_data.at[id,'blockchain']] if slippage_data.at[id,'blockchain'] in data['platforms'].keys() else data['symbol'].upper() for id,data in coin_data.iterrows()]
result['blockchain_with_highest_liq'] = [slippage_data.at[id,'blockchain'] for id,data in coin_data.iterrows()]
result = result[result['weight'] > 0]
result = result.sort_values("market_cap",ascending=False)
result

,name,market_cap,price,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,,
shiba-inu,Shiba Inu,4972947129,0.000008,0.333333,85,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
pepe,Pepe,477683449,0.000001,0.333333,85,0x6982508145454ce325ddbe47a25d4ec3d2311933,ethereum
floki,FLOKI,308865125,0.000031,0.333333,85,0xcf0c122c6b73ff809c693db761e7baebe62b6a2e,ethereum
